# Cruzamento de dados para RMSP

## Importações

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
import geopandas as gpd
import pandas as pd
from datetime import datetime

#from shapely import LineString, Polygon, MultiPolygon, distance, intersects, minimum_bounding_radius as min_radius
#from shapely.geometry import box
#from shapely.wkt import loads, dumps

In [2]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [3]:
nome_dir = '2000-2010-2022-RMSP-Dados'
if not os.path.isdir(nome_dir):
    os.mkdir(nome_dir)

In [4]:
UTMCRS = 'EPSG:31983'

dic_meses = {
    'jan':1,
    'fev':2,
    'mar':3,
    'abr':4,
    'mai':5,
    'jun':6,
    'jul':7,
    'ago':8,
    'set':9,
    'out':10,
    'nov':11,
    'dez':12,
    'feb':2,
    'apr':4,
    'may':5,
    'aug':8,
    'sep':9,
    'oct':10,
    'dec':12
}

### Leitura dos arquivos base

In [ ]:
gdf = gpd.read_file('2000-2010-2022/perimetros_compativeis_agg.gpkg')
df_A = pd.read_csv('2000-2010-2022/matriz_compat_2000.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
df_B = pd.read_csv('2000-2010-2022/matriz_compat_2010.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
df_C = pd.read_csv('2000-2010-2022/matriz_compat_2022.csv', sep='\t', dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})

In [ ]:
muns = ['3547304',
'3516309',
'3515707',
'3530607',
'3505708',
'3503901',
'3522505',
'3547809',
'3548807',
'3548708',
'3506607',
'3556453',
'3522208',
'3523107',
'3513009',
'3543303',
'3513801',
'3518305',
'3550308',
'3544103',
'3539103',
'3545001',
'3552502',
'3549953',
'3525003',
'3518800',
'3528502',
'3534401',
'3515103',
'3509007',
'3552809',
'3509205',
'3529401',
'3539806',
'3515004',
'3526209',
'3546801',
'3510609',
'3516408',]

In [ ]:
gdf['CD_MUN'] = gdf['CD_PERIMETRO'].apply(lambda x: x[:7])
gdf = gdf.query('CD_MUN in @muns')

## 1. Agregação com mancha urbana

In [ ]:
# Importação
mancha_2000 = gpd.read_file('../../Expansão Metropolitana/Mancha Urbana/GPKG/Coleção 8/MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_1985-2022_SIRGAS2000.gpkg', layer='MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_2000_SIRGAS2000')
mancha_2010 = gpd.read_file('../../Expansão Metropolitana/Mancha Urbana/GPKG/Coleção 8/MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_1985-2022_SIRGAS2000.gpkg', layer='MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_2010_SIRGAS2000')
mancha_2022 = gpd.read_file('../../Expansão Metropolitana/Mancha Urbana/GPKG/Coleção 8/MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_1985-2022_SIRGAS2000.gpkg', layer='MapBiomas_ManchaUrbana_Corr_SemRodov_RMSP_2022_SIRGAS2000')

In [ ]:
# Dissolução
mancha_2000_d = mancha_2000.dissolve()
mancha_2010_d = mancha_2010.dissolve()
mancha_2022_d = mancha_2022.dissolve()

In [ ]:
# Cálculo da área
gdf['area_ha'] = round(gdf.area/10000,3)

# Cálculo da área urbana
intersecao = gpd.overlay(gdf, mancha_2000, how='intersection')
intersecao['area_urb_ha_2000'] = round(intersecao.area/10000,3)
intersecao = intersecao.pivot_table(index='CD_PERIMETRO', values='area_urb_ha_2000', aggfunc='sum').reset_index()
df_mancha = gdf.merge(intersecao, on='CD_PERIMETRO', how='left').fillna(0)

intersecao = gpd.overlay(gdf, mancha_2010, how='intersection')
intersecao['area_urb_ha_2010'] = round(intersecao.area/10000,3)
intersecao = intersecao.pivot_table(index='CD_PERIMETRO', values='area_urb_ha_2010', aggfunc='sum').reset_index()
df_mancha = df_mancha.merge(intersecao, on='CD_PERIMETRO', how='left').fillna(0)

intersecao = gpd.overlay(gdf, mancha_2022, how='intersection')
intersecao['area_urb_ha_2022'] = round(intersecao.area/10000,3)
intersecao = intersecao.pivot_table(index='CD_PERIMETRO', values='area_urb_ha_2022', aggfunc='sum').reset_index()
df_mancha = df_mancha.merge(intersecao, on='CD_PERIMETRO', how='left').fillna(0)

In [ ]:
df_mancha = df_mancha[['CD_PERIMETRO','area_ha','area_urb_ha_2000','area_urb_ha_2010','area_urb_ha_2022']]
df_mancha.to_csv(f'{nome_dir}/mancha_urbana.csv', index=False, sep='\t', decimal=',')

## 2.1. Agregação com expansão urbana

In [ ]:
# Importação
expansao = gpd.read_file('../../Expansão Metropolitana/Perímetros de Expansão/Coleção 8/Análise Censo 2010-2022/MapBiomas_PerimetrosExp_Corr_ComRodov_RMSP_2010-2022_SIRGAS2000.gpkg')
expansao = expansao[['geometry', 'tipo']]

In [ ]:
# Exclusão de pixels soltos
expansao['geometry'] = expansao.buffer(2)
expansao = gpd.GeoDataFrame(expansao, geometry='geometry', crs=UTMCRS)

expansao = expansao.dissolve(by='tipo', as_index=False)
expansao = expansao.explode()
expansao = expansao[expansao.area >= 1000]

expansao['geometry'] = expansao.buffer(-2)
expansao = gpd.GeoDataFrame(expansao, geometry='geometry', crs=UTMCRS)

In [ ]:
# Cálculo da área urbana
intersecao = gpd.overlay(gdf, expansao, how='intersection')
intersecao['area_ha'] = round(intersecao.area/10000,3)

intersecao = intersecao.pivot_table(index='CD_PERIMETRO', columns='tipo', values='area_ha', aggfunc='sum').reset_index()
df_expansao = gdf.merge(intersecao, on='CD_PERIMETRO', how='left').fillna(0)

In [ ]:
df_expansao = gdf.merge(intersecao, on='CD_PERIMETRO', how='left').fillna(0)

# Renomear
rename_cols = {
    'fav / ocup / lot': 'fav_ocup_lot',
    'hor_outro': 'horz',
    'galp / log / ind': 'ind_galp',
    'conj hab / cond vert': 'conj_vert',
    'rod': 'rodovia',
}
df_expansao = df_expansao.rename(columns=rename_cols)

In [ ]:
df_expansao = df_expansao[['CD_PERIMETRO', 'cond_hor', 'conj_cond_vert', 'fav_ocup_lot', 'ind_galp', 'hor', 'outro', 'rodovia', 'vert']]
df_expansao = df_expansao.set_index('CD_PERIMETRO')
df_expansao['area_expansao_ha'] = df_expansao.sum(axis=1)
df_expansao = df_expansao.reset_index()
df_expansao = df_expansao.rename(columns={k:f'expansao_ha_{k.replace(" ","_")}' for k in ['cond_hor', 'conj_cond_vert', 'fav_ocup_lot', 'ind_galp', 'hor', 'outro', 'rodovia', 'vert']})

In [ ]:
df_expansao.to_csv(f'{nome_dir}/expansao.csv', index=False, sep='\t', decimal=',')

## 3. Agregação com dados de Aglomerados Subnormais 2010

In [ ]:
agsn = pd.read_csv('lista_agsn_2010.txt', sep='\t', dtype={'CD_SETOR':str,'CD_AGSN':str})
agsn = agsn.rename(columns={'CD_SETOR':'CD_GEOCODI'})
agsn = agsn[['CD_GEOCODI', 'CD_AGSN']]

# Merge e reagregação
df_agsn = df_B.merge(agsn, on='CD_GEOCODI', how='left')
df_agsn = df_agsn.rename(columns={'CD_AGSN':'CD_AGSN_2010'})
df_agsn['AGSN_2010'] = df_agsn['CD_AGSN_2010'].fillna('').apply(lambda x: True if len(x)>0 else False)

In [ ]:
# Exportação
df_agsn[['CD_PERIMETRO','AGSN_2010']].drop_duplicates().to_csv(f'{nome_dir}/agsn.csv', index=False, sep='\t', decimal=',')

## 4. Agregação com dados Embraesp

In [ ]:
# Importação
emb1 = gpd.read_file('../Residencial_Embraesp_06-22.gpkg')
emb1 = emb1.to_crs(UTMCRS)
emb2 = gpd.read_file('../Residencial_Embraesp_RMSP_06-22.gpkg')
emb2 = emb2.to_crs(UTMCRS)

embraesp_lab = pd.concat([emb1, emb2])
embraesp_lab = embraesp_lab[['geometry', 'DATA_ENT', 'AU_EMP', 'TIPO_EMP', 'TT_UNID', 'GAR_UNID']]

In [ ]:
# Atribuir setor censitário de 2010
malha_ibge_2010 = gpd.read_file('Setores IBGE.gpkg', layer='SP_2010')
malha_ibge_2010['CD_MUN'] = malha_ibge_2010['CD_GEOCODI'].apply(lambda x: x[:7])
malha_ibge_2010 = malha_ibge_2010.query('CD_MUN in @muns')
malha_ibge_2010 = malha_ibge_2010.to_crs(UTMCRS)
malha_ibge_2010 = malha_ibge_2010[['geometry', 'CD_GEOCODI']]

# Corrigir data entrega
embraesp_lab['DATA_ENT'] = embraesp_lab['DATA_ENT'].apply(lambda x: datetime(int('20'+x.split('/')[-1]), dic_meses[x.split('/')[-2]], 1))

In [ ]:
embraesp_lab = gpd.overlay(embraesp_lab, malha_ibge_2010, how='union')\
                .dropna(subset=['CD_GEOCODI'])

In [ ]:
# Importar dados CEM
embraesp_cem = gpd.read_file('../../../dados/CEM Embraesp/LanRes_85_13_RMSP_CEM.shp')
embraesp_cem = embraesp_cem.to_crs(UTMCRS)
embraesp_cem['CD_GEOCODI'] = embraesp_cem['SC_2010'].astype(int).astype(str)

In [ ]:
embraesp_cem['DATA_ENT'] = embraesp_cem['DATA_ENT'].apply(lambda x: x.lower())
embraesp_cem['DATA_ENT'] = embraesp_cem['DATA_ENT'].apply(lambda x: datetime(int(x.split('-')[-1]), dic_meses[x.split('-')[-2]], 1))
embraesp_cem = embraesp_cem.query('ANO_LAN < 2006')
embraesp_cem = embraesp_cem[['AU_EMP', 'DATA_ENT', 'TIPO_EMP', 'TT_UNID', 'GAR_UNID', 'CD_GEOCODI']]

In [ ]:
embraesp = pd.concat([embraesp_lab, embraesp_cem])

In [ ]:
# Seleção de dados no período entre censos
dataref1 = datetime(2000, 8, 1)
dataref2 = datetime(2022, 8, 1)
embraesp = embraesp.query('DATA_ENT > @dataref1 & DATA_ENT <= @dataref2')
embraesp = embraesp[['AU_EMP', 'DATA_ENT', 'TT_UNID', 'GAR_UNID', 'CD_GEOCODI']]
embraesp['PERIODO'] = embraesp['DATA_ENT'].apply(lambda x: '00a10' if x <= datetime(2010,8,1) else '10a22')
embraesp['GAR'] = embraesp['TT_UNID']*embraesp['GAR_UNID']

In [ ]:
df_embraesp = embraesp.merge(df_B, on='CD_GEOCODI')
df_embraesp['AU_EMP'] = df_embraesp['AU_EMP'].apply(lambda x: float(x.replace(',','.')) if isinstance(x, str) else x)
df_embraesp = df_embraesp.pivot_table(index='CD_PERIMETRO', columns='PERIODO', values=['AU_EMP', 'TT_UNID', 'GAR'], aggfunc='sum').reset_index()
df_embraesp.columns = ['_'.join(col).strip() for col in df_embraesp.columns.values]
df_embraesp = df_embraesp.rename(columns={'CD_PERIMETRO_':'CD_PERIMETRO'})
df_embraesp = df_embraesp.fillna(0)

In [ ]:
# Exportação
df_embraesp.to_csv(f'{nome_dir}/embraesp.csv', index=False, sep='\t', decimal=',')

## 5. Áreas Near Transit

In [ ]:
# Leitura
area_nt = gpd.read_file('../TRANSPORTE/area_near_transit.gpkg')
area_nt = area_nt.explode()
area_nt['NEAR_TRANSIT'] = True

In [ ]:
# Interseção
geodata = gdf[['geometry', 'CD_PERIMETRO']]
geodata['AREA_ORIGINAL'] = geodata.area
area_nt = gpd.overlay(geodata[['geometry', 'CD_PERIMETRO', 'AREA_ORIGINAL']], area_nt, how='intersection')
area_nt = area_nt.dropna(subset=['CD_PERIMETRO'])

area_nt = area_nt[area_nt['AREA_ORIGINAL']*0.5 <= area_nt.area]
area_nt = area_nt.drop_duplicates()

In [ ]:
# Exportação
area_nt[['CD_PERIMETRO','NEAR_TRANSIT']].to_csv(f'{nome_dir}/near_transit.csv', index=False, sep='\t', decimal=',')

## 5. Junção

In [5]:
# Reimportar gdf
gdf = gpd.read_file('2000-2010-2022/perimetros_compativeis_agg.gpkg')

# Mesclagem
for file in ['mancha_urbana', 'embraesp', 'agsn', 'near_transit', 'expansao']:
    df_temp = pd.read_csv(f'{nome_dir}/{file}.csv', sep='\t', decimal=',', dtype={'CD_PERIMETRO':str})
    gdf = gdf.merge(df_temp, on='CD_PERIMETRO', how='left')

In [6]:
gdf = gdf.rename(columns={k:k.upper() for k in gdf.columns if k!='geometry'})

In [7]:
gdf = gdf.drop_duplicates(subset='CD_PERIMETRO')
gdf.to_file(f'{nome_dir}/PICs_RMSP_001022_agg.gpkg', layer='PICs_RMSP_001022', driver='GPKG')